* 得到本月总体销量
* 总体销量时间窗口特征
* 总体车型时间窗口特征
* 测试 LightGBM

In [1]:
import numpy as np
import scipy as sp

import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('../../raw/LiChuan/dataset.csv')
submit = pd.read_csv('../../raw/CarsSaleForecast/yancheng_testA_20171225.csv')

In [3]:
data['sale_date'] = data['year'] * 100 + data['month']
data['sale_date'] = data['sale_date'].astype(int)
data['sale_date'] = pd.to_datetime(data['sale_date'],format='%Y%m',errors = "coerce")
data['sale_date'] = data['sale_date'].dt.to_period('m')

In [4]:
# 得到本月销量总和
sale_quantity_total_this_month = data.groupby(data['sale_date']).sale_quantity.sum().reset_index()

data = pd.merge(data, sale_quantity_total_this_month, on='sale_date', how='left')

data.rename(columns={'sale_quantity_y': 'sale_quantity_total_this_month'}, inplace=True) # 替换列名称
data.rename(columns={'sale_quantity_x': 'sale_quantity'}, inplace=True) # 替换列名称

In [5]:
def sale_cal_sum(data,window,num):
    datatmp = pd.DataFrame()
    for i in range(num):          
        dataSaletmp = data[data['sale_date'].isin(window)]
        dataSaletmp = dataSaletmp.loc[:,['class_id', 'sale_quantity_total_this_month', 'this_month_sale_class_count']]
        dataSaletmp = dataSaletmp.groupby(['class_id']).agg(sum)[['sale_quantity_total_this_month', 'this_month_sale_class_count']].reset_index() #更换所需的函数
        dataSaletmp['sale_date'] = window[0]+1
        if i == 0:
            datatmp = dataSaletmp
        else:
            datatmp = pd.concat([datatmp, dataSaletmp])    
        for j,k in enumerate(window):
            window[j] = window[j] + 1
    return datatmp

In [6]:
def sale_cal_max(data,window,num):
    datatmp = pd.DataFrame()
    for i in range(num):          
        dataSaletmp = data[data['sale_date'].isin(window)]
        dataSaletmp = dataSaletmp.loc[:,['class_id','sale_quantity_total_this_month']]
        dataSaletmp = dataSaletmp.groupby(['class_id']).agg(max).sale_quantity_total_this_month.reset_index() #更换所需的函数
        dataSaletmp['sale_date'] = window[0]+1
        if i == 0:
            datatmp = dataSaletmp
        else:
            datatmp = pd.concat([datatmp, dataSaletmp])    
        for j,k in enumerate(window):
            window[j] = window[j] + 1
    return datatmp

In [7]:
def sale_cal_min(data,window,num):
    datatmp = pd.DataFrame()
    for i in range(num):          
        dataSaletmp = data[data['sale_date'].isin(window)]
        dataSaletmp = dataSaletmp.loc[:,['class_id','sale_quantity_total_this_month']]
        dataSaletmp = dataSaletmp.groupby(['class_id']).agg(min).sale_quantity_total_this_month.reset_index() #更换所需的函数
        dataSaletmp['sale_date'] = window[0]+1
        if i == 0:
            datatmp = dataSaletmp
        else:
            datatmp = pd.concat([datatmp, dataSaletmp])    
        for j,k in enumerate(window):
            window[j] = window[j] + 1
    return datatmp

In [8]:
def sale_cal_mean(data,window,num):
    datatmp = pd.DataFrame()
    for i in range(num):          
        dataSaletmp = data[data['sale_date'].isin(window)]
        dataSaletmp = dataSaletmp.loc[:,['class_id','sale_quantity_total_this_month']]
        dataSaletmp = dataSaletmp.groupby(['class_id']).mean().sale_quantity_total_this_month.reset_index() #更换所需的函数
        dataSaletmp['sale_date'] = window[0]+1
        if i == 0:
            datatmp = dataSaletmp
        else:
            datatmp = pd.concat([datatmp, dataSaletmp])    
        for j,k in enumerate(window):
            window[j] = window[j] + 1
    return datatmp

In [9]:
def sale_cal_std(data,window,num):
    datatmp = pd.DataFrame()
    for i in range(num):          
        dataSaletmp = data[data['sale_date'].isin(window)]
        dataSaletmp = dataSaletmp.loc[:,['class_id','sale_quantity_total_this_month']]
        dataSaletmp = dataSaletmp.groupby(['class_id']).std().sale_quantity_total_this_month.reset_index() #更换所需的函数
        dataSaletmp['sale_date'] = window[0]+1
        if i == 0:
            datatmp = dataSaletmp
        else:
            datatmp = pd.concat([datatmp, dataSaletmp])    
        for j,k in enumerate(window):
            window[j] = window[j] + 1
    return datatmp

## 滑动2个月数据

In [10]:
i = data['sale_date'][0]+1  #从2012-2开始计算

In [11]:
window=[i,i-1] #每次计算必须重新初始化window
data_last_2M_sum = sale_cal_sum(data,window,69) #计算近2个月的数据总共需要计算64个月
data_last_2M_sum.rename(columns={'sale_quantity_total_this_month': 'sale_quantity_total_last_2M_sum', 'this_month_sale_class_count': 'last_2M_sale_class_count_sum'}, inplace=True)# 替换列名称

window=[i,i-1] #每次计算必须重新初始化window
data_last_2M_max = sale_cal_max(data,window,69)
data_last_2M_max.rename(columns={'sale_quantity_total_this_month': 'sale_quantity_total_last_2M_max', 'this_month_sale_class_count': 'last_2M_sale_class_count_max'}, inplace=True)# 替换列名称

window=[i,i-1] #每次计算必须重新初始化window
data_last_2M_min = sale_cal_min(data,window,69)
data_last_2M_min.rename(columns={'sale_quantity_total_this_month': 'sale_quantity_total_last_2M_min', 'this_month_sale_class_count': 'last_2M_sale_class_count_min'}, inplace=True)# 替换列名称

window=[i,i-1] #每次计算必须重新初始化window
data_last_2M_mean = sale_cal_mean(data,window,69)
data_last_2M_mean.rename(columns={'sale_quantity_total_this_month': 'sale_quantity_total_last_2M_mean', 'this_month_sale_class_count': 'last_2M_sale_class_count_mean'}, inplace=True)# 替换列名称

window=[i,i-1] #每次计算必须重新初始化window
data_last_2M_std = sale_cal_std(data,window,69)
data_last_2M_std.rename(columns={'sale_quantity_total_this_month': 'sale_quantity_total_last_2M_std', 'this_month_sale_class_count': 'last_2M_sale_class_count_std'}, inplace=True)# 替换列名称

In [12]:
# 合并
data = pd.merge(data, data_last_2M_sum, on=['class_id','sale_date'], how='left')
data = pd.merge(data, data_last_2M_max, on=['class_id','sale_date'], how='left')
data = pd.merge(data, data_last_2M_min, on=['class_id','sale_date'], how='left')
data = pd.merge(data, data_last_2M_mean, on=['class_id','sale_date'], how='left')
data = pd.merge(data, data_last_2M_std, on=['class_id','sale_date'], how='left')
data.fillna(0, inplace=True)

## 滑动 3 个月数据

In [13]:
i = data['sale_date'][0]+2  #从2012-4开始计算

In [14]:
window=[i,i-1,i-2] #每次计算必须重新初始化window
data_last_3M_sum = sale_cal_sum(data,window,68) #计算近2个月的数据总共需要计算64个月
data_last_3M_sum.rename(columns={'sale_quantity_total_this_month': 'sale_quantity_total_last_3M_sum', 'this_month_sale_class_count': 'last_3M_sale_class_count_sum'}, inplace=True)# 替换列名称

window=[i,i-1,i-2] #每次计算必须重新初始化window
data_last_3M_max = sale_cal_max(data,window,68)
data_last_3M_max.rename(columns={'sale_quantity_total_this_month': 'sale_quantity_total_last_3M_max', 'this_month_sale_class_count': 'last_3M_sale_class_count_max'}, inplace=True)# 替换列名称

window=[i,i-1,i-2] #每次计算必须重新初始化window
data_last_3M_min = sale_cal_min(data,window,68)
data_last_3M_min.rename(columns={'sale_quantity_total_this_month': 'sale_quantity_total_last_3M_min', 'this_month_sale_class_count': 'last_3M_sale_class_count_min'}, inplace=True)# 替换列名称

window=[i,i-1,i-2] #每次计算必须重新初始化window
data_last_3M_mean = sale_cal_mean(data,window,68)
data_last_3M_mean.rename(columns={'sale_quantity_total_this_month': 'sale_quantity_total_last_3M_mean', 'this_month_sale_class_count': 'last_3M_sale_class_count_mean'}, inplace=True)# 替换列名称

window=[i,i-1,i-2] #每次计算必须重新初始化window
data_last_3M_std = sale_cal_std(data,window,68)
data_last_3M_std.rename(columns={'sale_quantity_total_this_month': 'sale_quantity_total_last_3M_std', 'this_month_sale_class_count': 'last_3M_sale_class_count_std'}, inplace=True)# 替换列名称

In [15]:
# 合并
data = pd.merge(data, data_last_3M_sum, on=['class_id','sale_date'], how='left')
data = pd.merge(data, data_last_3M_max, on=['class_id','sale_date'], how='left')
data = pd.merge(data, data_last_3M_min, on=['class_id','sale_date'], how='left')
data = pd.merge(data, data_last_3M_mean, on=['class_id','sale_date'], how='left')
data = pd.merge(data, data_last_3M_std, on=['class_id','sale_date'], how='left')
data.fillna(0, inplace=True)

## 滑动 6 个月数据

In [16]:
i = data['sale_date'][0]+5  #从2012-7开始计算

In [17]:
window=[i,i-1,i-2,i-3,i-4,i-5] #每次计算必须重新初始化window
data_last_6M_sum = sale_cal_sum(data,window,65) #计算近2个月的数据总共需要计算64个月
data_last_6M_sum.rename(columns={'sale_quantity_total_this_month': 'sale_quantity_total_last_6M_sum', 'this_month_sale_class_count': 'last_6M_sale_class_count_sum'}, inplace=True)# 替换列名称

window=[i,i-1,i-2,i-3,i-4,i-5] #每次计算必须重新初始化window
data_last_6M_max = sale_cal_max(data,window,65)
data_last_6M_max.rename(columns={'sale_quantity_total_this_month': 'sale_quantity_total_last_6M_max', 'this_month_sale_class_count': 'last_6M_sale_class_count_max'}, inplace=True)# 替换列名称

window=[i,i-1,i-2,i-3,i-4,i-5] #每次计算必须重新初始化window
data_last_6M_min = sale_cal_min(data,window,65)
data_last_6M_min.rename(columns={'sale_quantity_total_this_month': 'sale_quantity_total_last_6M_min', 'this_month_sale_class_count': 'last_6M_sale_class_count_min'}, inplace=True)# 替换列名称

window=[i,i-1,i-2,i-3,i-4,i-5] #每次计算必须重新初始化window
data_last_6M_mean = sale_cal_mean(data,window,65)
data_last_6M_mean.rename(columns={'sale_quantity_total_this_month': 'sale_quantity_total_last_6M_mean', 'this_month_sale_class_count': 'last_6M_sale_class_count_mean'}, inplace=True)# 替换列名称

window=[i,i-1,i-2,i-3,i-4,i-5] #每次计算必须重新初始化window
data_last_6M_std = sale_cal_std(data,window,65)
data_last_6M_std.rename(columns={'sale_quantity_total_this_month': 'sale_quantity_total_last_6M_std', 'this_month_sale_class_count': 'last_6M_sale_class_count_std'}, inplace=True)# 替换列名称

In [18]:
# 合并
data = pd.merge(data, data_last_6M_sum, on=['class_id','sale_date'], how='left')
data = pd.merge(data, data_last_6M_max, on=['class_id','sale_date'], how='left')
data = pd.merge(data, data_last_6M_min, on=['class_id','sale_date'], how='left')
data = pd.merge(data, data_last_6M_mean, on=['class_id','sale_date'], how='left')
data = pd.merge(data, data_last_6M_std, on=['class_id','sale_date'], how='left')
data.fillna(0, inplace=True)

## 滑动 12 个月数据

In [19]:
i = data['sale_date'][0]+11  #从2013-1开始计算

In [20]:
window=[i,i-1,i-2,i-3,i-4,i-5,i-6,i-7,i-8,i-9,i-10,i-11] #每次计算必须重新初始化window
data_last_12M_sum = sale_cal_sum(data,window,59) #计算近2个月的数据总共需要计算64个月
data_last_12M_sum.rename(columns={'sale_quantity_total_this_month': 'sale_quantity_total_last_12M_sum', 'this_month_sale_class_count': 'last_12M_sale_class_count_sum'}, inplace=True)# 替换列名称

window=[i,i-1,i-2,i-3,i-4,i-5,i-6,i-7,i-8,i-9,i-10,i-11] #每次计算必须重新初始化window
data_last_12M_max = sale_cal_max(data,window,59)
data_last_12M_max.rename(columns={'sale_quantity_total_this_month': 'sale_quantity_total_last_12M_max', 'this_month_sale_class_count': 'last_12M_sale_class_count_max'}, inplace=True)# 替换列名称

window=[i,i-1,i-2,i-3,i-4,i-5,i-6,i-7,i-8,i-9,i-10,i-11] #每次计算必须重新初始化window
data_last_12M_min = sale_cal_min(data,window,59)
data_last_12M_min.rename(columns={'sale_quantity_total_this_month': 'sale_quantity_total_last_12M_min', 'this_month_sale_class_count': 'last_12M_sale_class_count_min'}, inplace=True)# 替换列名称

window=[i,i-1,i-2,i-3,i-4,i-5,i-6,i-7,i-8,i-9,i-10,i-11] #每次计算必须重新初始化window
data_last_12M_mean = sale_cal_mean(data,window,59)
data_last_12M_mean.rename(columns={'sale_quantity_total_this_month': 'sale_quantity_total_last_12M_mean', 'this_month_sale_class_count': 'last_12M_sale_class_count_mean'}, inplace=True)# 替换列名称

window=[i,i-1,i-2,i-3,i-4,i-5,i-6,i-7,i-8,i-9,i-10,i-11] #每次计算必须重新初始化window
data_last_12M_std = sale_cal_std(data,window,59)
data_last_12M_std.rename(columns={'sale_quantity_total_this_month': 'sale_quantity_total_last_12M_std', 'this_month_sale_class_count': 'last_12M_sale_class_count_std'}, inplace=True)# 替换列名称

In [21]:
# 合并
data = pd.merge(data, data_last_12M_sum, on=['class_id','sale_date'], how='left')
data = pd.merge(data, data_last_12M_max, on=['class_id','sale_date'], how='left')
data = pd.merge(data, data_last_12M_min, on=['class_id','sale_date'], how='left')
data = pd.merge(data, data_last_12M_mean, on=['class_id','sale_date'], how='left')
data = pd.merge(data, data_last_12M_std, on=['class_id','sale_date'], how='left')
data.fillna(0, inplace=True)

In [22]:
data = data.drop(['sale_quantity_total_this_month', 'sale_quantity_total_last_year'], axis=1)

In [23]:
data.to_csv('../../raw/LiChuan/TotalDataSet.csv',index=False, header=True)

## 测试 LightGBM

In [24]:
# 去掉 2012 年数据
data = data[data['year'] != 2012]
labels = data['sale_quantity']
data = data.drop(['sale_date', 'class_id', 'sale_quantity'], axis=1)

In [25]:
# 训练集 2013-01 至 2017-07, 测试集 2017-08
train_1 = data[:-560]
test_1 = data[-560:-420]
labels_1 = labels[:-560]
test_labels_1 = labels[-560:-420]

# 训练集 2013-01 至 2017-08, 测试集 2017-09
train_2 = data[:-420]
test_2 = data[-420:-280]
labels_2 = labels[:-420]
test_labels_2 = labels[-420:-280]

# 训练集 2013-01 至 2017-09, 测试集 2017-10
train_3 = data[:-280]
test_3 = data[-280:-140]
labels_3 = labels[:-280]
test_labels_3 = labels[-280:-140]

In [26]:
lgb_train_1 = lgb.Dataset(train_1, labels_1)

params = {
'learning_rate': 0.002,
'boosting_type': 'gbdt',
'objective': 'regression',
'metric': 'rmse',
'sub_feature': 0.8,
'num_leaves': 35,
'min_data': 20,
'min_hessian': 1,
'verbose': -1,}

model_1 = lgb.train(params, lgb_train_1, 900)
lgb_pred_1 = model_1.predict(test_1)
rmsetmp_1 = sp.sqrt(sp.mean((test_labels_1 - lgb_pred_1) ** 2))

print('This time rmse is: '+ str(rmsetmp_1))

This time rmse is: 123.216579106


In [27]:
lgb_train_2 = lgb.Dataset(train_2, labels_2)

params = {
'learning_rate': 0.002,
'boosting_type': 'gbdt',
'objective': 'regression',
'metric': 'rmse',
'sub_feature': 0.8,
'num_leaves': 35,
'min_data': 20,
'min_hessian': 1,
'verbose': -1,}

model_2 = lgb.train(params, lgb_train_2, 900)
lgb_pred_2 = model_2.predict(test_2)
rmsetmp_2 = sp.sqrt(sp.mean((test_labels_2 - lgb_pred_2) ** 2))

print('This time rmse is: '+ str(rmsetmp_2))

This time rmse is: 121.880387939


In [28]:
lgb_train_3 = lgb.Dataset(train_3, labels_3)

params = {
'learning_rate': 0.003,
'boosting_type': 'gbdt',
'objective': 'regression',
'metric': 'rmse',
'sub_feature': 0.8,
'num_leaves': 35,
'min_data': 30,
'min_hessian': 1,
'verbose': -1,}

model_3 = lgb.train(params, lgb_train_3, 900)
lgb_pred_3 = model_3.predict(test_3)
rmsetmp_3 = sp.sqrt(sp.mean((test_labels_3 - lgb_pred_3) ** 2))

print('This time rmse is: '+ str(rmsetmp_3))

This time rmse is: 145.407142519


In [29]:
print('The average rmse is: '+ str((rmsetmp_1 + rmsetmp_2 + rmsetmp_3)/3))

The average rmse is: 130.168036521


* 废了好大功夫, 反而下降了